<a href="https://colab.research.google.com/github/SaeidDaliriSusefi/Air-Temperature-Monitoring/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install xee -q
!pip install pymannkendall -q

In [3]:
import ee
import geemap
import xarray as xr
import xee
import pymannkendall as mk
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import FormatStrFormatter
import json
import geopandas as gpd
import pandas as pd

In [4]:
ee.Authenticate()
ee.Initialize(project="Your google earth engine project name", opt_url='https://earthengine-highvolume.googleapis.com')

In [8]:
# Select the area of intrest
map=geemap.Map(basemap="SATELLITE")
map

In [6]:
roi = map.draw_last_feature.geometry()
start_time = '1980'
end_time = '2025'


collection = ee.ImageCollection("ECMWF/ERA5/MONTHLY").select('mean_2m_air_temperature').filterDate(start_time,end_time)
ds = xr.open_dataset(collection, engine = 'ee', crs = 'EPSG:4326', scale = 0.27, geometry = roi)


In [7]:
def mk_test(img):
  test = mk.original_test(img)
  trend = test.trend
  trend_class = {'increasing':1, 'decreasing': -1, 'no_trend':0}
  trend_reclass = trend_class.get(trend, 0)
  score = test.s
  pval = test.p
  return trend_reclass, score, pval


change_trend, change_score, change_pval = xr.apply_ufunc(
    mk_test,
    ds.mean_2m_air_temperature,
    input_core_dims = [['time']],
    output_core_dims = [[],[],[]],
    dask = 'allowed',
    vectorize = True,
    output_dtypes = ['int','float64', 'float64']
)

change_trend = change_trend.rename('trend')
change_score = change_score.rename('score') * 0.0001   # just to normalize the scallbar
change_pval = change_pval.rename('p_value')


In [8]:
Country_name = 'Iran'
Country = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq('country_na', Country_name))
iran_geojson = Country.geometry().getInfo()
iran_geojson_dict = json.dumps(iran_geojson)
iran_gdf = gpd.read_file(iran_geojson_dict)


min_lon, min_lat, max_lon, max_lat = iran_gdf.total_bounds

fig, axes = plt.subplots(1, 3, figsize=(25, 25), constrained_layout=False)
titles = ['Trend', 'Score', 'P-value']
datasets = [change_trend, change_score, change_pval]
cmaps = ['coolwarm', 'viridis', 'viridis']
vmins = [-1, None, None]
vmaxs = [1, None, 0.05]


cmap_score = mcolors.LinearSegmentedColormap.from_list('green_orange_red', ['green', 'orange', 'red'])


cmap_pvalue = mcolors.LinearSegmentedColormap.from_list('reverse_pvalue', ['pink', 'white', 'grey'])

cmap_trend = mcolors.ListedColormap(['blue', 'gray', 'red'])


norm_trend = mcolors.BoundaryNorm(boundaries=[-1, -0.5, 0.5, 1], ncolors=3)

for idx, (ax, data, title, cmap, vmin, vmax) in enumerate(zip(axes, datasets, titles, cmaps, vmins, vmaxs)):
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    if title == 'Score':
        im = data.plot(
            x='lon', y='lat', ax=ax, add_colorbar=False,
            cmap=cmap_score, vmin=0, vmax=1, robust=True
        )
        ax.set_title('Intensity')
    elif title == 'P-value':
        im = data.plot(
            x='lon', y='lat', ax=ax, add_colorbar=False,
            cmap=cmap_pvalue, vmin=0, vmax=0.05, robust=True
        )
        ax.set_title('Accuracy')
    else:
        im = data.plot(
            x='lon', y='lat', ax=ax, add_colorbar=False,
            cmap=cmap_trend, norm=norm_trend, robust=True
        )
        ax.set_title('Trend')

    iran_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=0.8)


    cbar = plt.colorbar(im, cax=cax, cmap=cmap)

    if title == 'Trend':
        cbar.set_ticks([-1, 0, 1])
        cbar.set_ticklabels(['Decreasing', 'No trend', 'Increasing'])
    elif title == 'Score':
        cbar.set_ticks([0, 0.5, 1])
        cbar.set_ticklabels(['Low', 'Moderate', 'High'])
    elif title == 'P-value':
        cbar.set_ticks([0, 0.05])
        cbar.set_ticklabels(['100%', '95%'])

    ax.set_xlim(min_lon, max_lon)
    ax.set_ylim(min_lat, max_lat)


plt.subplots_adjust(hspace=0.3)


plt.savefig('output_plot.png', dpi=300, bbox_inches='tight')


plt.show()
